In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
#offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
#inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [4]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [5]:
assortment_info_df_30 = assortment_info_df[(assortment_info_df['offer_times']>=30)]
assortments_30 = []
for i in range(len(assortment_info_df_30['assortments'])):
    assortments_30.append(assortment_info_df_30['assortments'][i])


jd_choice_30 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_30:
        jd_choice_30.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_30 = assortment_info_df_30['assortments']
choice_collection_30 = np.zeros((n,len(collection_30)))

frequency_collection_30 = np.zeros((n,len(collection_30)))

purchaseset_list_30 = jd_choice_30['purchaseset'].tolist()

inc_prod_num_30 = assortment_info_df_30['includ_prod_num']
choice_prob_30 = jd_choice_30['choice prob']
frequency_30 = jd_choice_30['purchasenum']

global_num = 0
for i in range(len(collection_30)):
    local_num =0
    while(local_num < inc_prod_num_30[i]):
        if purchaseset_list_30[global_num][0] in assortments_30[i]:
            choice_collection_30[purchaseset_list_30[global_num][0]][i] = choice_prob_30[global_num]
            frequency_collection_30[purchaseset_list_30[global_num][0]][i] = frequency_30[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_30[global_num])
            print(assortments_30[i])
            print('error!')
            break
#print(global_num)  

choice_check_30 = pd.DataFrame(choice_collection_30)
choice_check_30.to_csv('instances/full_instances/full_30.csv')
frequency_collection_30 = frequency_collection_30.astype(int)
frequency_check_30 = pd.DataFrame(frequency_collection_30)
frequency_check_30.to_csv('instances/full_instances/frequency_30.csv')

offer_times_30 = assortment_info_df_30['offer_times'].tolist()
print('the number of assortments with at least 30 purchases is',len(assortments_30))

the number of assortments with at least 30 purchases is 24


In [6]:
assortment_info_df_40 = assortment_info_df[(assortment_info_df['offer_times']>=40)]
assortments_40 = []
for i in range(len(assortment_info_df_40['assortments'])):
    assortments_40.append(assortment_info_df_40['assortments'][i])


jd_choice_40 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_40:
        jd_choice_40.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_40 = assortment_info_df_40['assortments']
choice_collection_40 = np.zeros((n,len(collection_40)))

frequency_collection_40 = np.zeros((n,len(collection_40)))

purchaseset_list_40 = jd_choice_40['purchaseset'].tolist()

inc_prod_num_40 = assortment_info_df_40['includ_prod_num']
choice_prob_40 = jd_choice_40['choice prob']
frequency_40 = jd_choice_40['purchasenum']

global_num = 0
for i in range(len(collection_40)):
    local_num =0
    while(local_num < inc_prod_num_40[i]):
        if purchaseset_list_40[global_num][0] in assortments_40[i]:
            choice_collection_40[purchaseset_list_40[global_num][0]][i] = choice_prob_40[global_num]
            frequency_collection_40[purchaseset_list_40[global_num][0]][i] = frequency_40[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_40[global_num])
            print(assortments_40[i])
            print('error!')
            break
#print(global_num)  

choice_check_40 = pd.DataFrame(choice_collection_40)
choice_check_40.to_csv('instances/full_instances/full_40.csv')
frequency_collection_40 = frequency_collection_40.astype(int)
frequency_check_40 = pd.DataFrame(frequency_collection_40)
frequency_check_40.to_csv('instances/full_instances/frequency_40.csv')

offer_times_40 = assortment_info_df_40['offer_times'].tolist()
print('the number of assortments with at least 40 purchases is',len(assortments_40))

the number of assortments with at least 40 purchases is 19


In [7]:
assortment_info_df_50 = assortment_info_df[(assortment_info_df['offer_times']>=50)]
assortments_50 = []
for i in range(len(assortment_info_df_50['assortments'])):
    assortments_50.append(assortment_info_df_50['assortments'][i])


jd_choice_50 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_50:
        jd_choice_50.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_50 = assortment_info_df_50['assortments']
choice_collection_50 = np.zeros((n,len(collection_50)))

frequency_collection_50 = np.zeros((n,len(collection_50)))

purchaseset_list_50 = jd_choice_50['purchaseset'].tolist()

inc_prod_num_50 = assortment_info_df_50['includ_prod_num']
choice_prob_50 = jd_choice_50['choice prob']
frequency_50 = jd_choice_50['purchasenum']

global_num = 0
for i in range(len(collection_50)):
    local_num =0
    while(local_num < inc_prod_num_50[i]):
        if purchaseset_list_50[global_num][0] in assortments_50[i]:
            choice_collection_50[purchaseset_list_50[global_num][0]][i] = choice_prob_50[global_num]
            frequency_collection_50[purchaseset_list_50[global_num][0]][i] = frequency_50[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_50[global_num])
            print(assortments_50[i])
            print('error!')
            break
#print(global_num)  

choice_check_50 = pd.DataFrame(choice_collection_50)
choice_check_50.to_csv('instances/full_instances/full_50.csv')
frequency_collection_50 = frequency_collection_50.astype(int)
frequency_check_50 = pd.DataFrame(frequency_collection_50)
frequency_check_50.to_csv('instances/full_instances/frequency_50.csv')

offer_times_50 = assortment_info_df_50['offer_times'].tolist()
print('the number of assortments with at least 50 purchases is',len(assortments_50))

the number of assortments with at least 50 purchases is 15


In [8]:
assortment_info_df_60 = assortment_info_df[(assortment_info_df['offer_times']>=60)]
assortments_60 = []
for i in range(len(assortment_info_df_60['assortments'])):
    assortments_60.append(assortment_info_df_60['assortments'][i])


jd_choice_60 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_60:
        jd_choice_60.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_60 = assortment_info_df_60['assortments']
choice_collection_60 = np.zeros((n,len(collection_60)))

frequency_collection_60 = np.zeros((n,len(collection_60)))

purchaseset_list_60 = jd_choice_60['purchaseset'].tolist()

inc_prod_num_60 = assortment_info_df_60['includ_prod_num']
choice_prob_60 = jd_choice_60['choice prob']
frequency_60 = jd_choice_60['purchasenum']

global_num = 0
for i in range(len(collection_60)):
    local_num =0
    while(local_num < inc_prod_num_60[i]):
        if purchaseset_list_60[global_num][0] in assortments_60[i]:
            choice_collection_60[purchaseset_list_60[global_num][0]][i] = choice_prob_60[global_num]
            frequency_collection_60[purchaseset_list_60[global_num][0]][i] = frequency_60[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_60[global_num])
            print(assortments_60[i])
            print('error!')
            break
#print(global_num)  

choice_check_60 = pd.DataFrame(choice_collection_60)
choice_check_60.to_csv('instances/full_instances/full_60.csv')
frequency_collection_60 = frequency_collection_60.astype(int)
frequency_check_60 = pd.DataFrame(frequency_collection_60)
frequency_check_60.to_csv('instances/full_instances/frequency_60.csv')

offer_times_60 = assortment_info_df_60['offer_times'].tolist()
print('the number of assortments with at least 60 purchases is',len(assortments_60))

the number of assortments with at least 60 purchases is 13


In [9]:
assortment_info_df_75 = assortment_info_df[(assortment_info_df['offer_times']>=75)]
assortments_75 = []
for i in range(len(assortment_info_df_75['assortments'])):
    assortments_75.append(assortment_info_df_75['assortments'][i])


jd_choice_75 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_75:
        jd_choice_75.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_75 = assortment_info_df_75['assortments']
choice_collection_75 = np.zeros((n,len(collection_75)))

frequency_collection_75 = np.zeros((n,len(collection_75)))

purchaseset_list_75 = jd_choice_75['purchaseset'].tolist()

inc_prod_num_75 = assortment_info_df_75['includ_prod_num']
choice_prob_75 = jd_choice_75['choice prob']
frequency_75 = jd_choice_75['purchasenum']

global_num = 0
for i in range(len(collection_75)):
    local_num =0
    while(local_num < inc_prod_num_75[i]):
        if purchaseset_list_75[global_num][0] in assortments_75[i]:
            choice_collection_75[purchaseset_list_75[global_num][0]][i] = choice_prob_75[global_num]
            frequency_collection_75[purchaseset_list_75[global_num][0]][i] = frequency_75[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_75[global_num])
            print(assortments_75[i])
            print('error!')
            break
#print(global_num)  

choice_check_75 = pd.DataFrame(choice_collection_75)
choice_check_75.to_csv('instances/full_instances/full_75.csv')
frequency_collection_75 = frequency_collection_75.astype(int)
frequency_check_75 = pd.DataFrame(frequency_collection_75)
frequency_check_75.to_csv('instances/full_instances/frequency_75.csv')

offer_times_75 = assortment_info_df_75['offer_times'].tolist()
print('the number of assortments with at least 75 purchases is',len(assortments_75))

the number of assortments with at least 75 purchases is 12


In [10]:
assortment_info_df_100 = assortment_info_df[(assortment_info_df['offer_times']>=100)]
assortments_100 = []
for i in range(len(assortment_info_df_100['assortments'])):
    assortments_100.append(assortment_info_df_100['assortments'][i])

jd_choice_100 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_100:
        jd_choice_100.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_100 = assortment_info_df_100['assortments']
choice_collection_100 = np.zeros((n,len(collection_100)))

frequency_collection_100 = np.zeros((n,len(collection_100)))

purchaseset_list_100 = jd_choice_100['purchaseset'].tolist()

inc_prod_num_100 = assortment_info_df_100['includ_prod_num']
choice_prob_100 = jd_choice_100['choice prob']
frequency_100 = jd_choice_100['purchasenum']

global_num=0
for i in range(len(collection_100)):
    local_num =0
    while(local_num < inc_prod_num_100[i]):
        if purchaseset_list_100[global_num][0] in assortments_100[i]:
            choice_collection_100[purchaseset_list_100[global_num][0]][i] = choice_prob_100[global_num]
            frequency_collection_100[purchaseset_list_100[global_num][0]][i] = frequency_100[global_num]
            global_num = global_num +1
            local_num = local_num +1

choice_check_100 = pd.DataFrame(choice_collection_100)
choice_check_100.to_csv('instances/full_instances/full_100.csv')
frequency_collection_100 = frequency_collection_100.astype(int)
frequency_check_100 = pd.DataFrame(frequency_collection_100)
frequency_check_100.to_csv('instances/full_instances/frequency_100.csv')

offer_times_100 = assortment_info_df_100['offer_times'].tolist()
print('the number of assortments with at least 100 purchases is',len(assortments_100))

the number of assortments with at least 100 purchases is 11


In [11]:
assortment_info_df_15 = assortment_info_df[(assortment_info_df['offer_times']>=15)]
assortments_15 = []
for i in range(len(assortment_info_df_15['assortments'])):
    assortments_15.append(assortment_info_df_15['assortments'][i])


jd_choice_15 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_15:
        jd_choice_15.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_15 = assortment_info_df_15['assortments']
choice_collection_15 = np.zeros((n,len(collection_15)))

frequency_collection_15 = np.zeros((n,len(collection_15)))

purchaseset_list_15 = jd_choice_15['purchaseset'].tolist()

inc_prod_num_15 = assortment_info_df_15['includ_prod_num']
choice_prob_15 = jd_choice_15['choice prob']
frequency_15 = jd_choice_15['purchasenum']

global_num = 0
for i in range(len(collection_15)):
    local_num =0
    while(local_num < inc_prod_num_15[i]):
        if purchaseset_list_15[global_num][0] in assortments_15[i]:
            choice_collection_15[purchaseset_list_15[global_num][0]][i] = choice_prob_15[global_num]
            frequency_collection_15[purchaseset_list_15[global_num][0]][i] = frequency_15[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_15[global_num])
            print(assortments_15[i])
            print('error!')
            break
#print(global_num)  

choice_check_15 = pd.DataFrame(choice_collection_15)
choice_check_15.to_csv('instances/full_instances/full_15.csv')
frequency_collection_15 = frequency_collection_15.astype(int)
frequency_check_15 = pd.DataFrame(frequency_collection_15)
frequency_check_15.to_csv('instances/full_instances/frequency_15.csv')

offer_times_15 = assortment_info_df_15['offer_times'].tolist()
print('the number of assortments with at least 15 purchases is',len(assortments_15))

the number of assortments with at least 15 purchases is 38


representability: even for the data set with assortments offered with 100 times, such a data set cannot be representable for the regular model, RUM and MDM cannot represent all the instances above

In [12]:
collection_100 = collection_100.tolist()
''' check regularity '''
flag = 0
for i in range(len(collection_100)):
    for j in range(i+1,len(collection_100)):
        if {*collection_100[i]} <= {*collection_100[j]}:
            for k in collection_100[i]:
                #if k !=0 and choice_collection_100[k][i] < choice_collection_100[k][j]:
                if choice_collection_100[k][i] < choice_collection_100[k][j]:
                    flag = 1
        if {*collection_100[i]} >= {*collection_100[j]}:
            for k in collection_100[j]:
                if choice_collection_100[k][j] < choice_collection_100[k][i]:
                #if k !=0 and choice_collection_100[k][j] < choice_collection_100[k][i]:
                    flag = 1
    if flag == 1:
        break
if flag == 0:
    print('REGULAR')
else:
    print('NOT REGURLAR')

NOT REGURLAR


In [13]:
## MDM limit formulation
def row_assortment_data(data,collection):
    pre_data = copy.deepcopy(data)
    
    for j in range(data.shape[0]):
        for i in range(len(collection)):
            if j not in collection[i]:
                pre_data[j][i] = -1
    
    return pre_data

def find_indexes_nonnegative_descending(arr):
    row_indexes_sorted_desc_nonnegative = []
    row_indexes_positive_same_value = []

    for row in arr:
        # Find indexes with non-negative values
        nonnegative_row = row[row >= 0]  # Consider only non-negative values
        unique_elements, unique_indexes = np.unique(nonnegative_row, return_index=True)
        sorted_indexes_desc = np.nonzero(row >= 0)[0][unique_indexes][np.argsort(-unique_elements)]

        row_indexes_sorted_desc_nonnegative.append(sorted_indexes_desc)

        # Keep all indexes of elements with positive and same values
        positive_values = unique_elements[unique_elements > 0]
        positive_same_value_indexes = [np.where(row == val)[0] for val in positive_values if np.count_nonzero(row == val) > 1]
        row_indexes_positive_same_value.append(positive_same_value_indexes)

    return row_indexes_sorted_desc_nonnegative, row_indexes_positive_same_value


In [14]:
def mdm_feasibility(data,ranking,equal):
      
    ub = 100

    model = gp.Model('mdm')
    #model.setParam(gp.GRB.Param.TimeLimit, 60)
    model.setParam('OutputFlag', 0)
    eps = model.addVar(name='eps')
    lam = model.addVars(data.shape[1],name = 'lam')
    
    model.addConstr(eps<=ub ) # just give an upper bound (+ve) for eps, o.w., the problem is unbounded
    
    #for i in range(ranking.shape[0]): # from each product 
    for i in range(len(ranking)): # from each product 
        for j in range(len(ranking[i])-1): # the ranking of assortment in product i 
            model.addConstr(lam [ranking[i][j+1]] - lam[ranking[i][j]]-  eps >=0)
    
    for i in range(len(equal)):
        if len(equal[i])>0:
            for j in range(len(equal[i][0])-1):
                model.addConstr(lam[equal[i][0][j]]-lam[equal[i][0][j+1]]==0)
    
    model.setObjective(eps,GRB.MAXIMIZE)
    model.optimize()
    
     # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        print('Optimal solution found!')
    

    if (model.objVal>0):
      
        return [1,model.Runtime]
    else:
       
        #return [0, used_time]
        return [0,model.Runtime]
    

In [15]:
def mdm_limit(data, collection,offer_times):
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    eps = 0.01

    model = gp.Model()
    model.setParam('OutputFlag', 0)
    lam = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0.0, name="lam")
    x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="x")
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta")
    
    # Define auxiliary binary variables for absolute value terms
    abs_vars = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="abs")
    ''' abs_vars = {}
    for i in range(n):
        for j in range(m):
            abs_vars[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"abs_{i}_{j}") '''

    # Add constraints
    for j in range(m):
        model.addConstr(sum(x[i, j] for i in range(n)) == 1, name=f"normalization_{j}")

    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                model.addConstr(x[j, i] == 0, name=f"forcing_zero_{j}_{0}")
                #model.addConstr(abs_vars[j, i] == 0, name=f"abs_forcing_zero_{j}_{0}")
            else:
                model.addConstr(x[j, i] - data[j, i] <= abs_vars[j, i], name=f"abs_const1_{j}_{i}")
                model.addConstr(-x[j, i] + data[j, i] <= abs_vars[j, i], name=f"abs_const2_{j}_{i}")
    
    for i in range(m):
        model.addConstr(lam[i] <= 1, name=f"lam_bound_{i}")

    for i in range(len(prod_in_ass)):
        if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"pro_{i}_aspair_{i}_{j}_{k}")
                        
                        model.addConstr(x[i, j] - x[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"ppro_{i}_aspair_{j}_{k}")

    # Add linear constraints to represent absolute value terms
    ''' for i in range(n):
        for j in range(m):
            model.addConstr(x[i, j] - data[i, j] <= abs_vars[i, j], name=f"abs_const1_{i}_{j}")
            model.addConstr(-x[i, j] + data[i, j] <= abs_vars[i, j], name=f"abs_const2_{i}_{j}") '''

    # Define the modified objective function using auxiliary variables
    obj = (sum(offer_times[j]*data[i, j] * abs_vars[i, j] for i in range(n) for j in range(m)))
    #obj = (sum(offer_times[j]*data[i][j] * abs_vars[i, j] for i in range(n) for j in range(m))/sum(offer_times))

    model.setObjective(obj, GRB.MINIMIZE)

    # Set Gurobi parameters if needed
    model.optimize()
    
    x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
    x_values = np.array(x_values)
    
    lam_values = [lam[i].x for i in range(m)]
    lam_values = np.array(lam_values)

    if model.Status == GRB.OPTIMAL:
        print("Gurobi optimization status:", model.status)
        return [model.objVal, model.Runtime,lam_values,x_values]
    else:
        print("Gurobi optimization status:", model.status)
        model.write("mdmlimit_model.lp") 
        #return [model.objVal, model.Runtime,lam_values,x_values]

In [16]:
mdm_limit_100 = mdm_limit(choice_collection_100, collection_100,offer_times_100)
mdm_limit_75 = mdm_limit(choice_collection_75, collection_75,offer_times_75)
mdm_limit_60 = mdm_limit(choice_collection_60, collection_60,offer_times_60)
mdm_limit_50 = mdm_limit(choice_collection_50, collection_50,offer_times_50)
mdm_limit_40 = mdm_limit(choice_collection_40, collection_40,offer_times_40)
mdm_limit_30 = mdm_limit(choice_collection_30, collection_30,offer_times_30)
mdm_limit_20 = mdm_limit(choice_collection_20, collection_20,offer_times_20)

Academic license - for non-commercial use only - expires 2024-12-10
Using license file /Users/autumn/gurobi.lic
Gurobi optimization status: 2
Gurobi optimization status: 2
Gurobi optimization status: 2
Gurobi optimization status: 2
Gurobi optimization status: 2
Gurobi optimization status: 2
Gurobi optimization status: 2


In [17]:
#mdm_limit_15 = mdm_limit(choice_collection_15, collection_15,offer_times_15)

In [18]:
#mdm_limit_15

In [19]:
offertime_list = [20,30,40,50,60,75,100]
mdm_limit_loss = [ mdm_limit_20[0],mdm_limit_30[0],mdm_limit_40[0],mdm_limit_50[0],mdm_limit_60[0],mdm_limit_75[0],mdm_limit_100[0]]
avg_mdm_limit_loss = [ mdm_limit_20[0]/np.sum(offer_times_20),
                      mdm_limit_30[0]/np.sum(offer_times_30), mdm_limit_40[0]/np.sum(offer_times_40),
                      mdm_limit_50[0]/np.sum(offer_times_50), mdm_limit_60[0]/np.sum(offer_times_60),
                      mdm_limit_75[0]/np.sum(offer_times_75),mdm_limit_100[0]/np.sum(offer_times_100)]
mdm_limit_runtime = [mdm_limit_20[1],mdm_limit_30[1],mdm_limit_40[1],mdm_limit_50[1],mdm_limit_60[1],mdm_limit_75[1],mdm_limit_100[1]]
mdm_limit_prob =  [mdm_limit_20[-1],mdm_limit_30[-1],mdm_limit_40[-1],mdm_limit_50[-1],mdm_limit_60[-1],mdm_limit_75[-1],mdm_limit_100[-1]]

In [20]:
jd_mdm_limit_summary = pd.DataFrame({'offertimes':offertime_list,'mdm_loss':mdm_limit_loss,'avg_mdm_loss':avg_mdm_limit_loss,
                                     'mdm_limit_runtime':mdm_limit_runtime})
jd_mdm_limit_summary.to_csv('limit/summary/mdm_limit_summary.csv')

In [21]:
jd_mdm_limit_summary

,offertimes,mdm_loss,avg_mdm_loss,mdm_limit_runtime
0,20,61.539667,0.008039,17.283415
1,30,55.567515,0.007380,6.143238
2,40,49.851186,0.006772,1.243742
3,50,37.014595,0.005148,0.418710
4,60,31.294445,0.004419,0.167107
5,75,30.078567,0.004288,0.184646
6,100,30.078567,0.004336,0.194294


In [22]:

for i in range(len(offertime_list)):
    limit_prob = mdm_limit_prob[i]
    df_limit_prob = pd.DataFrame(limit_prob)
    df_limit_prob.to_csv('limit/probability/mdm/prob_offertimes'+str(offertime_list[i])+'.csv')

In [23]:
mdm_1norm_lambda = [ mdm_limit_20[2],mdm_limit_30[2],mdm_limit_40[2],mdm_limit_50[2],mdm_limit_60[2],mdm_limit_75[2],mdm_limit_100[2]]
for i in range(len(offertime_list)):
    cur_lam = mdm_1norm_lambda[i]
    df_cur_lam = pd.DataFrame(cur_lam)
    df_cur_lam.to_csv('limit/probability/mdm/lambda/lam'+str(offertime_list[i])+'.csv')